## Problem 9

In [1]:
import numpy as np
from numpy.linalg import inv
from numpy import dot, matmul
import pandas as pd
from scipy import stats
import pprint

In [2]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.tsatools import lagmat
from statsmodels.stats.diagnostic import het_arch, het_white

In [3]:
data = pd.read_csv("Data_PG_UN.csv")

In [4]:
data.head()

,Date,PG,UN,HH,MKT,RF
0,1994-01-31,0.008772,0.004329,0.0247,0.0312,0.0025
1,1994-02-28,0.016397,0.020473,-0.0121,-0.0235,0.0021
2,1994-03-31,-0.019355,-0.061245,-0.0504,-0.0451,0.0027
3,1994-04-30,-0.078947,-0.087739,0.0164,0.0095,0.0027
4,1994-05-31,0.082419,0.092478,0.0092,0.0089,0.0031


In [5]:
def capm(y, x, rf):
    # ols reg
    n = y.shape[0]
    k = x.shape[1]
    ols_y = y - rf
    ols_x = np.concatenate((np.ones((n, 1)), x - matmul(rf, np.ones((1, k)) ) ), axis = 1).reshape(n, k+1)
#     print(ols_x.shape, matmul(rf, np.ones((1, k))).shape, x.shape )
    
    aux = inv( 
        dot(ols_x.T, ols_x)
    ) 
    ols_coeff = dot(
        dot(aux, ols_x.T),
        ols_y
    )
    ols_y_hat = dot(
        ols_x,
        ols_coeff
    )
#     print(dot(aux, ols_x.T).shape, ols_y.shape)
#     print(aux.shape, ols_coeff.shape, ols_y_hat.shape)
    alpha = ols_coeff[0]
    beta = ols_coeff[1:]
#     print(alpha, beta)
    
    # r2
    y_mean = np.mean(ols_y)
    tot = np.sum( (ols_y - y_mean) ** 2 )
    reg = np.sum( (ols_y - ols_y_hat) ** 2 )
    res = np.sum( (ols_y_hat - y_mean) ** 2 )
    r_squared = 1 - res / tot
    r_squared_adjusted = 1 - res * (n - 1) / (tot * (n - k - 1)) 
#     print(r_squared, r_squared_adjusted)
    
    # variance-covariance matrix, under homoskedastic
    expected_res_squared = np.mean( (ols_y_hat - ols_y) ** 2 )
    matrix_var_cov = aux * expected_res_squared
    
    # Wald test
    coeff_var = np.diag(matrix_var_cov)
    coeff_std = np.sqrt(coeff_var) 
    alpha_std = coeff_std[0]
    beta_std = coeff_std[1:]
    t_value = (beta * 1.0 - .0) / beta_std 
    
    ### Heterskedastic
    # access whether there is signifiant evidence for heterskedastic
#     print((ols_y_hat - ols_y).shape)
    White_stat = het_white(ols_y_hat - ols_y, ols_x)
    ARCH_stat = het_arch((ols_y_hat - ols_y).reshape(-1))
    
    # confidence interval
    confidence_interval = {
        "90_percent": (beta - 1.645 * beta_std, beta + 1.645 * beta_std),
        "95_percent": (beta - 1.960 * beta_std, beta + 1.960 * beta_std),
        "99_percent": (beta - 2.576 * beta_std, beta + 2.576 * beta_std)
    }
    
    # variance-covariance matrix under heterskedastic
    matrix_D = np.diag( ((ols_y_hat - ols_y) ** 2).reshape(-1) )
    matrix_var_cov_heter = dot(
        dot(
            aux,
            dot(
                dot(ols_x.T, matrix_D),
                ols_x
            )
        ),
        aux
    )
    
    # gls reg
    gls_aux = inv( 
        dot(
            dot(
                ols_x.T,
                matrix_D
            ),
            ols_x
        )
    ) 
    gls_coeff = dot(
        dot(
            dot(gls_aux, ols_x.T), 
            matrix_D
        ),
        ols_y
    )
    gls_alpha = gls_coeff[0]
    gls_beta = gls_coeff[1:]
    
    # confidence interval under heterskedastic
    coeff_var_heter = np.diag(matrix_var_cov_heter)
    coeff_std_heter = np.sqrt(coeff_var_heter)
    alpha_std_heter = coeff_std_heter[0]
    beta_std_heter = coeff_std_heter[1:]
#     print(beta_std_heter, alpha_std_heter)
    
    confidence_interval_heter = {
        "90_percent": (beta - 1.645 * beta_std_heter, beta + 1.645 * beta_std_heter),
        "95_percent": (beta - 1.960 * beta_std_heter, beta + 1.960 * beta_std_heter),
        "99_percent": (beta - 2.576 * beta_std_heter, beta + 2.576 * beta_std_heter)
    }
    
    # Wald test
    t_value_heter = (beta * 1.0 - .0) / beta_std_heter
    
    ### Others
    # AIC, BIC and HQIC
    logLikelihood =  -0.5 * n * (np.log(2 * np.pi) + np.log(np.sum(beta_std ** 2)) + 1)
    AIC = k * 2.0 / n - 2 * logLikelihood / n  
    BIC = k * np.log(n) * 2.0 / n - 2 * logLikelihood / n
    HQIC = k * np.log(np.log(n)) * 2.0 / n - 2 * logLikelihood / n
    
    # DW stat, BG stat
    DW_stat = durbin_watson(ols_y_hat - ols_y)
    BG_stat = breusch_godfrey(ols_y_hat - ols_y, ols_x)
    
    # JB stat
    JB_stat = jarque_bera(ols_y_hat - ols_y)
    
    ### Muticolinearity
    # cond number
    condition_number = np.linalg.cond(dot(ols_x.T, ols_x))
    
    # Ramsey's RESET test
    Ramsey_RESET = reset_ramsey(ols_y_hat - ols_y, ols_x, ols_y, ols_y_hat)
    
    # scatter plot 
    rtn = {
        "sample_size": n,
        "regressors": k,
        "alpha": alpha,
        "beta": beta,
        "coeff": ols_coeff,
        "gls_alpha": gls_alpha,
        "gls_beta": gls_beta,
        "gls_coeff": gls_coeff,
        
        "r_squared": r_squared,
        "r_squared_adjusted": r_squared_adjusted,
        
        "matrix_var_cov": matrix_var_cov,
        
        "t_value": t_value,
        
        "White_stat": White_stat,
        "ARCH_stat": ARCH_stat,
        
        "confidence_interval": confidence_interval,
        
        "matrix_var_cov_heter": matrix_var_cov_heter,
        
        "confidence_interval_heter": confidence_interval_heter,
        
        "t_value_heter": t_value_heter,
        
        "AIC": AIC,
        "BIC": BIC,
        "HQIC": HQIC,
        
        "DW_stat": DW_stat,
        "BG_stat": BG_stat,
        
        "JB_stat": JB_stat,
        
        "condition_number": condition_number,
        
        "Ramset_RESET": Ramsey_RESET
    }
    return rtn
    
    
    

In [6]:
def durbin_watson(resids, axis=0):
    """
    return DW_stat
    """
    resids = np.asarray(resids)
    diff_resids = np.diff(resids, 1, axis=axis)
    dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
    return dw

In [7]:
def jarque_bera(resids, axis=0):
    """
    return:
    jb_test, jb_pv, skew, kurtosis
    """
    resids = np.asarray(resids)
    # Calculate residual skewness and kurtosis
    skew = stats.skew(resids, axis=axis)
    kurtosis = 3 + stats.kurtosis(resids, axis=axis)

    # Calculate the Jarque-Bera test for normality
    n = resids.shape[axis]
    jb = (n / 6.) * (skew ** 2 + (1 / 4.) * (kurtosis - 3) ** 2)
    jb_pv = stats.chi2.sf(jb, 2)

    return jb, jb_pv, skew, kurtosis

In [8]:
def breusch_godfrey(resids, exog, nlags=None):
    '''
    return
    BG stats
    '''
    x = np.asarray(resids).reshape(-1)
    exog_old = exog
    nobs = x.shape[0]
    if nlags is None:
        nlags = np.trunc(12. * np.power(nobs/100., 1/4.))#nobs//4  #TODO: check default, or do AIC/BIC
        nlags = int(nlags)

    x = np.concatenate((np.zeros(nlags), x))


    xdall = lagmat(x[:,None], nlags, trim='both')
    nobs = xdall.shape[0]
    xdall = np.c_[np.ones((nobs,1)), xdall]
    xshort = x[-nobs:]
    exog = np.column_stack((exog_old, xdall))
    k_vars = exog.shape[1]

    resols = OLS(xshort, exog).fit()
    ft = resols.f_test(np.eye(nlags, k_vars, k_vars - nlags))
    fval = ft.fvalue
    fpval = ft.pvalue
    fval = np.squeeze(fval)[()]   #TODO: fix this in ContrastResults
    fpval = np.squeeze(fpval)[()]
    lm = nobs * resols.rsquared
    lmpval = stats.chi2.sf(lm, nlags)

    return lm, lmpval, fval, fpval

In [9]:
def reset_ramsey(res, exog, ols_y, ols_y_hat, degree=5):
    '''
    RESET Ramsey stats
    '''
    order = degree + 1
    k_vars = exog.shape[1]

    y_fitted_vander = np.vander(ols_y_hat.reshape(-1), order)[:, :-2] #drop constant
    exog = np.column_stack((exog, y_fitted_vander))
    res_aux = OLS(ols_y, exog).fit()
    #r_matrix = np.eye(degree, exog.shape[1], k_vars)
    r_matrix = np.eye(degree-1, exog.shape[1], k_vars)

    return res_aux.f_test(r_matrix) #, r_matrix, res_aux

#### Question 1

In [10]:
y = np.array(data[["UN"]])
x = np.array(data[["MKT"]])
rf = np.array(data[["RF"]])
rtn = capm(y, x, rf)
pprint.pprint(rtn)

{'AIC': -1.8839579062540148,
 'ARCH_stat': (19.60818832755058,
               0.2383882398775724,
               1.2395068671945766,
               0.23899189833901005),
 'BG_stat': (23.12959631091408,
             0.08142679372129774,
             1.5812127465772183,
             0.0792652454067308),
 'BIC': -1.849291625169573,
 'DW_stat': array([2.31394602]),
 'HQIC': -1.8785150081814894,
 'JB_stat': (array([29.59168708]),
             array([3.75186148e-07]),
             array([-0.17047603]),
             array([4.60433945])),
 'Ramset_RESET': <class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1.6189636]]), p=0.16986196930817227, df_denom=258, df_num=4>,
 'White_stat': (6.198067177610722,
                0.04509275949070361,
                3.1374775116036404,
                0.045033179154105346),
 'alpha': array([0.00804347]),
 'beta': array([[0.0524295]]),
 'coeff': array([[0.00804347],
       [0.0524295 ]]),
 'condition_number': 514.8175504461651,
 'confide

#### Question 2

In [11]:
y = np.array(data[["UN"]])
x = np.array(data[["MKT", "HH"]])
rf = np.array(data[["RF"]])
rtn = capm(y, x, rf)
pprint.pprint(rtn)

{'AIC': -0.6476473406639189,
 'ARCH_stat': (21.06123882371936,
               0.17616860892935352,
               1.3398840900574611,
               0.1741835404834825),
 'BG_stat': (21.52682217040645,
             0.12082465246320606,
             1.4559956146686803,
             0.12246909030461185),
 'BIC': -0.5783147784950353,
 'DW_stat': array([2.2815962]),
 'HQIC': -0.6367615445188679,
 'JB_stat': (array([29.2676887]),
             array([4.41166117e-07]),
             array([-0.15356408]),
             array([4.60198978])),
 'Ramset_RESET': <class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[2.67729464]]), p=0.03234991695953332, df_denom=257, df_num=4>,
 'White_stat': (10.413815731591601,
                0.064323941206679,
                2.119014856035554,
                0.06360504458525529),
 'alpha': array([0.00824356]),
 'beta': array([[ 0.09675669],
       [-0.07486515]]),
 'coeff': array([[ 0.00824356],
       [ 0.09675669],
       [-0.07486515]]),
 'c

#### Question 3

- I prefer first model since its conditional number is smaller and adjusted r_squared is higher, which means the estimators are good and the dependent variables can explain the independent variables well.

#### Question 4

- The current market beta for UL, under the CAPM, is is beta estimate of first model 0.0524